#Integrate SimpleHTR with word beam search
Using CTCWordBeamSearch


In [ ]:
!rm -rf CTCWordBeamSearch

In [ ]:
!git clone https://github.com/githubharald/CTCWordBeamSearch
%cd ./CTCWordBeamSearch
!pip install .
%cd /content

Cloning into 'CTCWordBeamSearch'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 340 (delta 19), reused 72 (delta 12), pack-reused 256
Receiving objects: 100% (340/340), 1.63 MiB | 1.69 MiB/s, done.
Resolving deltas: 100% (116/116), done.
/content/CTCWordBeamSearch
Processing /content/CTCWordBeamSearch
  Created wheel for word-beam-search: filename=word_beam_search-1.0.0-cp37-cp37m-linux_x86_64.whl size=1178370 sha256=fa21298c74c98bc7021165862824783b4385c01e22e0f8c572c169fdcb1bc995
  Stored in directory: /tmp/pip-ephem-wheel-cache-emcrthjs/wheels/a9/69/4c/9d6acbecc7bf4b47c5072b213d9b08e4b9c43864bbed5206cc
Successfully built word-beam-search
/content


#Infer the model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/TFG/SimpleHTR/requirements.txt' '.'
!pip install -r requirements.txt
!pip install path

     |████████████████████████████████| 184kB 10.7MB/s 
     |████████████████████████████████| 286kB 16.6MB/s 
     |████████████████████████████████| 12.4MB 224kB/s 
     |████████████████████████████████| 49.5MB 62kB/s 
     |████████████████████████████████| 394.7MB 46kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: editdistance 0.5.3
    Uninstalling editdistance-0.5.3:
      Successfully uninstalled editdistance-0.5.3
  Found existing installation: lmdb 0.99
    Uninstalling lmdb-0.99:
      Successfully uninstalled lmdb-0.99
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow

In [ ]:
!cp '/content/drive/MyDrive/TFG/HTG/output.zip' '.'
!unzip './output.zip'
 
!cp -r '/content/drive/MyDrive/TFG/SimpleHTR/model' './model'
!cp -r '/content/drive/MyDrive/TFG/SimpleHTR/data' './data'
!cp -r '/content/drive/MyDrive/TFG/SimpleHTR/src' './src'

Se han truncado las últimas 5000 líneas del flujo de salida.
 extracting: content/output/line165/word38_img4.png  
  inflating: content/output/line165/word18_img4.png  
  inflating: content/output/line165/word24_img3.png  
 extracting: content/output/line165/word39_img1.png  
 extracting: content/output/line165/word27_img0.png  
  inflating: content/output/line165/word26_img4.png  
 extracting: content/output/line165/word5_img2.png  
  inflating: content/output/line165/word23_img3.png  
  inflating: content/output/line165/word9_img4.png  
  inflating: content/output/line165/word8_img4.png  
 extracting: content/output/line165/word18_img3.png  
  inflating: content/output/line165/word2_img0.png  
 extracting: content/output/line165/word6_img3.png  
  inflating: content/output/line165/word8_img3.png  
 extracting: content/output/line165/word6_img2.png  
 extracting: content/output/line165/word35_img3.png  
 extracting: content/output/line165/word14_img2.png  
  inflating: content/output/

In [ ]:
%cd ./src
!mkdir errors

/content/src
mkdir: cannot create directory ‘errors’: File exists


In [ ]:
import os
from openpyxl import Workbook
 
import argparse
import json
 
import cv2
import editdistance
from path import Path
 
from DataLoaderIAM import DataLoaderIAM, Batch
from Model import Model, DecoderType
from SamplePreprocessor import preprocess
 
from tqdm import tqdm
 
DECODER_TYPE = DecoderType.WordBeamSearch # DecoderType.BestPath    OR    DecoderType.BeamSearch    OR    DecoderType.WordBeamSearch
NUM_IMGS_PER_WORD = 5
#NUM_LINES = 445
 
class FilePaths:
    "filenames and paths to data"
    fnCharList = '../model/charList.txt'
    fnSummary = '../model/summary.json'
    fnInfer = '../content/output/'    # Output folder from the HTG
    fnCorpus = '../data/corpusGood2.txt' # corpus.txt is the original, corpusGood2.txt is the modified version
 
 
 
def infer(model, fnImg):
    "recognize text in image provided by file path"
    img = preprocess(cv2.imread(fnImg, cv2.IMREAD_GRAYSCALE), Model.imgSize)
    batch = Batch(None, [img])
    (recognized, probability) = model.inferBatch(batch, True)
    #print(f'Recognized: "{recognized[0]}"')
    #print(f'Probability: {probability[0]}')
 
    return (recognized, probability)
 
 
def main():
 
    # infer text on test image
    model = Model(open(FilePaths.fnCharList).read(), DECODER_TYPE, mustRestore=True, dump=False)
    
    wb = Workbook() # Work with .xlsx files (Excel)
    #imgsdict = {"0":"A", "1":"B", "2":"C", "3":"D", "4":"E"}
    err = 0
    # Loop on all images calling infer() to get the output
    for root, dirs, files in os.walk(FilePaths.fnInfer):
        for directory in tqdm(dirs):
            err_dir = ""
            wr = wb.create_sheet(directory)
            wp = wb.create_sheet(directory + "_prob")
            for _, _, files in os.walk(FilePaths.fnInfer + directory):
                cellsR = [[0 for i in range(NUM_IMGS_PER_WORD)] for j in range(len(files)//NUM_IMGS_PER_WORD)]
                cellsP = [[0 for i in range(NUM_IMGS_PER_WORD)] for j in range(len(files)//NUM_IMGS_PER_WORD)]
                for file in files:
                    word = file.split('_')[0][4:]
                    img  = file.split('_')[1][3:].split('.')[0]
                    try:
                        recprob = infer(model, FilePaths.fnInfer + directory + "/" + file) # Infer the image and get the recognized and probability as a tuple
                    except:
                        err += 1
                        err_dir = err_dir + file + ' '
                        # print("Error in line: ", directory, file, "Total errors: ", err)
                        raise
                    try:
                      cellsR [int(word)][int(img)] = recprob[0][0]
                      cellsP [int(word)][int(img)] = recprob[1][0]
                    except:
                      print("ERROR IN IMAGE!! ")
                      print(directory, word, img, str(len(cellsR)), str(len(cellsR[0])), recprob)
                      err_dir = err_dir + file + ' '
                      pass
                for w in range(len(cellsR)):
                    wr.append(cellsR[w])
                    wp.append(cellsP[w])
                wb.save("inferOutput.xlsx")
            if err_dir != "":
              with open("./errors/" +directory + "_errors.txt", "w") as erf:
                erf.write(err_dir)
        break
    print("FINISHED. TOTAL ERRORS: ", err)
 
 
if __name__ == '__main__':
    main()

In [ ]:
 
from google.colab import files
!zip -r errors.zip errors
files.download("errors.zip")
!cp 'errors.zip' '/content/drive/MyDrive/TFG/SimpleHTR/errors.zip'

In [ ]:
from google.colab import files
files.download("inferOutput.xlsx")
 
!cp 'inferOutput.xlsx' '/content/drive/MyDrive/TFG/SimpleHTR/inferOutput.xlsx'